In [3]:
##
import os
import numpy as np
import librosa
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, LSTM, 
    Bidirectional, BatchNormalization, GlobalAveragePooling1D, Attention
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, LSTM, 
    Bidirectional, TimeDistributed, BatchNormalization, GlobalAveragePooling2D,GlobalAveragePooling1D
)

In [4]:
##
def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    return audio + noise_factor * noise

def shift_pitch(audio, n_steps=2):
    return librosa.effects.pitch_shift(audio, sr=22050, n_steps=n_steps)  # Default sample rate is 22050 Hz

def time_stretch(audio, rate=1.25):
    return librosa.effects.time_stretch(audio, rate=rate)

def extract_features(audio_file, augment=False):
    y, sr = librosa.load(audio_file, sr=None)
    if augment:
        augment_choice = np.random.choice(['noise', 'pitch', 'stretch', None])
        if augment_choice == 'noise':
            y = add_noise(y)
        elif augment_choice == 'pitch':
            y = shift_pitch(y)  
        elif augment_choice == 'stretch':
            y = time_stretch(y)

    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, max(0, 128 - mel_spec_db.shape[1]))), mode='constant')[:, :128]

    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)

    features = np.concatenate([ 
        mel_spec_db.flatten(),
        mfccs.mean(axis=1),
        chroma.mean(axis=1),
        spec_contrast.mean(axis=1),
        zero_crossing_rate.mean(axis=1)
    ])
    print(audio_file)
    return features

In [5]:
##
# Paths
base_path = r"C:\Users\hp\Downloads\audiodata"
train_real_path = os.path.join(base_path, 'new_train/real')
train_fake_path = os.path.join(base_path, 'new_train/fake')
test_real_path = os.path.join(base_path, 'eval/real')
test_fake_path = os.path.join(base_path, 'eval/fake')

# Gather data
train_real_files = [os.path.join(train_real_path, file) for file in os.listdir(train_real_path) if file.endswith('.wav')]
train_fake_files = [os.path.join(train_fake_path, file) for file in os.listdir(train_fake_path) if file.endswith('.wav')]
test_real_files = [os.path.join(test_real_path, file) for file in os.listdir(test_real_path) if file.endswith('.wav')]
test_fake_files = [os.path.join(test_fake_path, file) for file in os.listdir(test_fake_path) if file.endswith('.wav')]

In [6]:
##
# Prepare dataset
X_train, y_train, X_test, y_test = [], [], [], []
for file in train_real_files[:1000]:
    X_train.append(extract_features(file))
    X_train.append(extract_features(file, augment=True))
    y_train.extend([0, 0])
for file in train_fake_files[:1000]:
    X_train.append(extract_features(file))
    X_train.append(extract_features(file, augment=True))
    y_train.extend([1, 1])
for file in test_real_files[:200]:
    X_test.append(extract_features(file))
    y_test.append(0)
for file in test_fake_files[:200]:
    X_test.append(extract_features(file))
    y_test.append(1)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = to_categorical(np.array(y_train), num_classes=2)
y_test = to_categorical(np.array(y_test), num_classes=2)

C:\Users\hp\Downloads\audiodata\new_train/real\afdvusntas.wav
C:\Users\hp\Downloads\audiodata\new_train/real\afdvusntas.wav
C:\Users\hp\Downloads\audiodata\new_train/real\akfmhpsggq.wav
C:\Users\hp\Downloads\audiodata\new_train/real\akfmhpsggq.wav
C:\Users\hp\Downloads\audiodata\new_train/real\akqllxlxsb.wav
C:\Users\hp\Downloads\audiodata\new_train/real\akqllxlxsb.wav
C:\Users\hp\Downloads\audiodata\new_train/real\amfdrorkqd.wav
C:\Users\hp\Downloads\audiodata\new_train/real\amfdrorkqd.wav
C:\Users\hp\Downloads\audiodata\new_train/real\anhgqbcmdu.wav
C:\Users\hp\Downloads\audiodata\new_train/real\anhgqbcmdu.wav
C:\Users\hp\Downloads\audiodata\new_train/real\anknluqkkp.wav
C:\Users\hp\Downloads\audiodata\new_train/real\anknluqkkp.wav
C:\Users\hp\Downloads\audiodata\new_train/real\anopkfhgfk.wav
C:\Users\hp\Downloads\audiodata\new_train/real\anopkfhgfk.wav
C:\Users\hp\Downloads\audiodata\new_train/real\arsgkutymm.wav
C:\Users\hp\Downloads\audiodata\new_train/real\arsgkutymm.wav
C:\Users

C:\Users\hp\Downloads\audiodata\new_train/real\A_066_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_066_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_067_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_067_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_068_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_068_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_069_0_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_069_0_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_070_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_070_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_071_0_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_071_0_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_072_0_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_072_0_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_073_0_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_073_0_A.wav
C:\Users\hp\Downloads\au

C:\Users\hp\Downloads\audiodata\new_train/real\A_1037_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1039_5_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1039_5_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_103_0_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_103_0_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1040_5_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1040_5_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1041_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1041_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1042_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1042_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1043_5_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1043_5_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1045_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1045_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1046_5_B.wav
C:\Users\h

C:\Users\hp\Downloads\audiodata\new_train/real\A_1106_5_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1106_5_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1107_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1107_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1108_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1108_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1109_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1109_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_110_0_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_110_0_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1110_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1110_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1111_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1111_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1113_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1113_5_D.wav
C:\Users\h

C:\Users\hp\Downloads\audiodata\new_train/real\A_1171_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1171_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1172_5_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1172_5_C.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1173_5_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1173_5_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1174_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1174_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1175_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1175_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1176_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1176_5_D.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1177_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1177_5_A.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1178_5_B.wav
C:\Users\hp\Downloads\audiodata\new_train/real\A_1178_5_B.wav
C:\Users

...

C:\Users\hp\Downloads\audiodata\new_train/fake\B_9909_20_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9910_05_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9910_05_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9911_0_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9911_0_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9912_10_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9912_10_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9913_20_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9913_20_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9914_05_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9914_05_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9915_10_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9915_10_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9916_5_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9916_5_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9917_10_C.

C:\Users\hp\Downloads\audiodata\new_train/fake\B_9975_05_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9975_05_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9976_15_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9976_15_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9977_10_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9977_10_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9978_10_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9978_10_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9979_0_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9979_0_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9980_05_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9980_05_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9981_20_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9981_20_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9982_15_C.wav
C:\Users\hp\Downloads\audiodata\new_train/fake\B_9982_15_

C:\Users\hp\Downloads\audiodata\eval/real\A_2146_15_C.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2147_15_D.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2148_15_B.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2149_15_B.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_214_0_B.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2150_15_A.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2151_15_A.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2151_20_D.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2152_20_C.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2153_20_B.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2154_20_C.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2155_20_B.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2156_20_B.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2157_20_C.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2158_20_B.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_2159_20_C.wav
C:\Users\hp\Downloads\audiodata\eval/real\A_215_0_A.wav
C:\Users\hp\Downlo

C:\Users\hp\Downloads\audiodata\eval/fake\C_06364_05_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06365_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06366_05_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06367_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06368_10_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06369_5_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06370_05_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06371_5_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06372_5_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06373_20_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06374_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06375_20_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06376_10_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06377_20_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06378_10_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06379_10_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06380_05_A.wav
C:\

C:\Users\hp\Downloads\audiodata\eval/fake\C_06506_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06507_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06508_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06509_20_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06510_15_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06511_20_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06512_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06513_20_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06514_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06515_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06516_05_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06517_15_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06518_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06519_20_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06520_10_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06521_0_A.wav
C:\Users\hp\Downloads\audiodata\eval/fake\C_06522_5_A.wav
C:\Use

In [15]:
##
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (14528, 16417)
y_train shape: (14528, 2)


In [29]:
##
def build_model():
    input_layer = Input(shape=(X_train.shape[1],))
    x = Dense(512, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(2, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [30]:
##
# Build, train, and save the model
model = build_model()
model.summary()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)
model.save('deepfake_audio_model_26.h5')


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 16417)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │       8,406,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,574,082 (32.71 MB)

 Trainable params: 8,572,290 (32.70 MB)

 Non-trainable params: 1,792 (7.00 KB)

Epoch 1/20
454/454 ━━━━━━━━━━━━━━━━━━━━ 36s 69ms/step - accuracy: 0.5534 - loss: 1.0041 - val_accuracy: 0.5025 - val_loss: 1.2634
Epoch 2/20
454/454 ━━━━━━━━━━━━━━━━━━━━ 29s 64ms/step - accuracy: 0.7292 - loss: 0.5653 - val_accuracy: 0.7875 - val_loss: 0.4997
Epoch 3/20
454/454 ━━━━━━━━━━━━━━━━━━━━ 29s 63ms/step - accuracy: 0.8371 - loss: 0.3737 - val_accuracy: 0.5475 - val_loss: 1.1451
Epoch 4/20
454/454 ━━━━━━━━━━━━━━━━━━━━ 29s 63ms/step - accuracy: 0.8546 - loss: 0.3305 - val_accuracy: 0.5000 - val_loss: 2.1219
Epoch 5/20
454/454 ━━━━━━━━━━━━━━━━━━━━ 29s 63ms/step - accuracy: 0.8253 - loss: 0.3889 - val_accuracy: 0.7250 - val_loss: 0.4718
Epoch 6/20
454/454 ━━━━━━━━━━━━━━━━━━━━ 29s 63ms/step - accuracy: 0.8911 - loss: 0.2615 - val_accuracy: 0.5825 - val_loss: 1.6006
Epoch 7/20
454/454 ━━━━━━━━━━━━━━━━━━━━ 29s 64ms/step - accuracy: 0.8665 - loss: 0.3086 - val_accuracy: 0.7375 - val_loss: 0.4665
Epoch 8/20
454/454 ━━━━━━━━━━━━━━━━━━━━ 29s 63ms/step - accuracy: 0.8676 - loss: 0.3114 - 

In [28]:
##
import os
import random
import numpy as np

def evaluate_model(eval_real_path, eval_fake_path, model_path='deepfake_audio_model_24.h5'):
    model = load_model(model_path)

    real_files = [os.path.join(eval_real_path, f) for f in os.listdir(eval_real_path) if f.endswith('.wav')]
    fake_files = [os.path.join(eval_fake_path, f) for f in os.listdir(eval_fake_path) if f.endswith('.wav')]

    real_files_sample = random.sample(real_files, min(100, len(real_files)))
    fake_files_sample = random.sample(fake_files, min(100, len(fake_files)))

    eval_files = real_files_sample + fake_files_sample
    true_labels = [0] * len(real_files_sample) + [1] * len(fake_files_sample)

    combined = list(zip(eval_files, true_labels))
    random.shuffle(combined)
    eval_files, true_labels = zip(*combined)

    predictions = []
    for file in eval_files:
        audio_features = extract_features(file)
        audio_features = audio_features.reshape((1, *audio_features.shape, 1))
        
        prediction = model.predict(audio_features)
        predicted_label = np.argmax(prediction, axis=1)[0]
        predictions.append(predicted_label)

    correct_predictions = sum(p == t for p, t in zip(predictions, true_labels))
    accuracy = correct_predictions / len(true_labels)

    print(f"Accuracy on evaluation data: {accuracy * 100:.2f}%")
    return accuracy

eval_real_path = r"C:\Users\hp\Downloads\audiodata\eval\real"
eval_fake_path = r"C:\Users\hp\Downloads\audiodata\eval\fake"

evaluate_model(eval_real_path, eval_fake_path)


C:\Users\hp\Downloads\audiodata\eval\real\C_5799_5_C.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_4018_10_A.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_5610_10_A.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_21172_05_A.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_14797_05_A.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_3504_20_D.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_27400_20_B.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_4246_05_D.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_16799_15_A.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_30910_20_D.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
C:\Users\hp\Downloads\audiodata\eva

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_1306_20_D.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_26646_5_B.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_2761_15_D.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_4310_05_A.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_2789_15_A.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_26555_5_B.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_29560_10_C.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
C:\Users\hp\Downloads\audiodata\eval\fake\C_11313_15_A.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_1204_20_B.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
C:\Users\hp\Downloads\audiodata\eval\real\C_2640_5_B.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
C

0.635